In [26]:
%matplotlib inline
import pandas as pd
import numpy as np
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
import matplotlib as mpl 
#mpl.use("module://mplcairo.qt") 
from mplcairo import operator_t
from mpl_toolkits.axes_grid1 import make_axes_locatable
import subprocess
from lib.funcs import *

In [27]:
## Get the names and open data
renda_path = ('./data/CENSO/PessoaRenda_PB.csv') 

data = pd.read_csv(renda_path,                  
                sep=';', 
                encoding="utf-8",
                dtype={'Cod_setor': str,
                      'V020': float,
                      'V021': float,
                      'V022': float},
                   na_values = ['X'],
                lineterminator='\n')

In [28]:
## Open SHP
setores = gpd.read_file('./data/shp/jp_shp.shp')
setores = setores.to_crs(epsg=3857)

In [29]:
border = setores
border['new_column'] = 0
border = border.dissolve(by='new_column')
border = border[['geometry']]
border['geometry'] = border['geometry'].buffer(0.0001)

In [30]:
# Select only the data we are interested in
#v001 = Moradores em domicilios particular e domicilios coletivos
renda = data[['Cod_setor', 'V020', 'V021', 'V022']]
renda = renda.rename(columns={'V020': 'num_pessoas',
                             'V021': 'num_pessoas_rendimento',
                             'V022': 'rendimento_nominal'})

setores = setores[['geometry', 'NM_BAIRRO', 'CD_GEOCODI']]

In [31]:
populacao = pd.read_csv('./data/POPULACAO/populacao_parse.csv',
                       dtype={'Cod_setor': str})

In [32]:
#populacao.set_index('Cod_setor', inplace=True)
#renda.set_index('Cod_setor', inplace=True)


In [33]:
renda = pd.merge(populacao, renda, on='Cod_setor')

In [34]:
renda['num_pessoas'] = renda['MORADORES']

In [35]:
renda.drop(['MORADORES'], axis=1, inplace=True)

In [36]:
renda

,Cod_setor,num_pessoas,num_pessoas_rendimento,rendimento_nominal
0,250010605000001,1528,712.0,304733.0
1,250010605000002,500,273.0,177732.0
2,250010605000003,889,484.0,310937.0
3,250010605000004,860,372.0,141471.0
4,250010605000005,667,326.0,143194.0
...,...,...,...,...
5543,251720905000004,345,183.0,81061.0
5544,251720905000005,1105,525.0,212375.0
5545,251740705000001,876,469.0,219933.0
5546,251740705000002,603,368.0,135068.0


In [37]:
# Set the same inde
renda.set_index('Cod_setor', inplace=True)
setores.set_index('CD_GEOCODI', inplace=True)

In [38]:
setores = setores.join(renda)

In [39]:
bairros = setores.dissolve(by='NM_BAIRRO', aggfunc='sum')

In [40]:
bairros_interesse = ['Torre', 'Tambauzinho', 'Expedicionários', 'Miramar', 'Pedro Gondim', 'Estados']
setores_recorte = setores.loc[setores['NM_BAIRRO'].isin(bairros_interesse)]

In [41]:
setores_recorte['renda_per_capita'] = setores_recorte.apply(get_renda_per_capita, axis=1)
setores_recorte['renda_por_trabalhador'] = setores_recorte.apply(get_renda_por_trabalhador, axis=1)
bairros['renda_per_capita'] = bairros.apply(get_renda_per_capita, axis=1)
bairros['renda_por_trabalhador'] = bairros.apply(get_renda_por_trabalhador, axis=1)

<ipython-input-41-4fc7d2baae89>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  setores_recorte['renda_per_capita'] = setores_recorte.apply(get_renda_per_capita, axis=1)
<ipython-input-41-4fc7d2baae89>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  setores_recorte['renda_por_trabalhador'] = setores_recorte.apply(get_renda_por_trabalhador, axis=1)


In [43]:
bairros

,geometry,num_pessoas,num_pessoas_rendimento,rendimento_nominal,renda_per_capita,renda_por_trabalhador
NM_BAIRRO,,,,,,
Aeroclube,"POLYGON ((-3878845.787 -792159.924, -3878970.3...",9649.0,5975.0,17493369.0,1812.972225,2927.760502
Altiplano Cabo Branco,"POLYGON ((-3876916.727 -797181.774, -3877032.3...",5233.0,2996.0,7316627.0,1398.170648,2442.131842
Alto do Céu,"POLYGON ((-3880526.984 -791742.159, -3880526.7...",16557.0,7110.0,4081383.0,246.504983,574.034177
Alto do Mateus,"POLYGON ((-3885968.582 -797638.000, -3886002.0...",16281.0,7785.0,5279507.0,324.274123,678.164033
Anatólia,"POLYGON ((-3878566.488 -797931.549, -3878554.7...",1162.0,761.0,1549911.0,1333.830465,2036.676741
...,...,...,...,...,...,...
Trincheiras,"POLYGON ((-3883573.691 -795946.209, -3883573.6...",6995.0,3421.0,2684683.0,383.800286,784.765566
Valentina,"POLYGON ((-3878549.633 -804234.168, -3878556.9...",22452.0,12371.0,11102453.0,494.497283,897.458007
Varadouro,"POLYGON ((-3883870.618 -794980.219, -3883870.6...",3720.0,1979.0,1427824.0,383.823656,721.487620


In [50]:
bairros_recorte = bairros.loc[bairros.index.isin(bairros_interesse)]
bairros_recorte = bairros_recorte.drop('geometry', axis=1)

,num_pessoas,num_pessoas_rendimento,rendimento_nominal,renda_per_capita,renda_por_trabalhador
NM_BAIRRO,,,,,
Estados,7458.0,4302.0,13819138.0,1852.928131,3212.258949
Expedicionários,3625.0,2156.0,4590862.0,1266.444690,2129.342301
Miramar,9500.0,5400.0,18806784.0,1979.661474,3482.737778
Pedro Gondim,3360.0,2077.0,5914438.0,1760.249405,2847.586904
Tambauzinho,4932.0,3074.0,9863042.0,1999.805758,3208.536760
Torre,15193.0,9412.0,13481797.0,887.368986,1432.405121


In [75]:
setores_recorte.to_file('./data/RENDA/setores_recorte.shp')
bairros.to_file('./data/RENDA/bairros.shp')

In [76]:
export(setores_recorte, 'renda_setores_recorte', 3857)
export(bairros, 'renda_bairros', 3857)